# Final Project: Euchre
Tyler Gibson & Carter Shavitz | CSC 5661

https://bicyclecards.com/how-to-play/euchre

In [3]:
import random

## Euchre Environment

In [ ]:
class Euchre:
    """
    A class to manage the euchre board.
    """

    def __init__(self, config: dict) -> None:
        """
        The constructor for the class.

        Parameters
        ----------
        config : dict
            The configuration parameters for euchre.
        """
        self.config = config
        self.score_to_win = self.config['score_to_win']  # 5, 7, or 10
        self.values = self.config['values']  # 9 - A that can include 7's and/or 8's

        self.num_players = 4
        self.suits = ['♠', '♥', '♦', '♣']
        self.deck = [f"{value}{suit}" for suit in self.suits for value in self.values]

    def deal_cards(self, distribution_pattern=(3, 2)):
        """
        
        Parameters
        ----------
        

        Returns
        -------
        """
        random.shuffle(self.deck)
        print(self.deck)
    
        players_hands = {
            'p1': [],
            'p2': [],
            'p3': [],
            'p4': []
        }

        for i in range(5):
            players_hands['p1'].append(self.deck[i*4])
            players_hands['p2'].append(self.deck[i*4+1])
            players_hands['p3'].append(self.deck[i*4+2])
            players_hands['p4'].append(self.deck[i*4+3])

        trump_card = self.deck[20] #retrieve top card of deck left

    
    #    deck_index = 0
    #    for count, player in enumerate(list(players_hands.keys()) * 2):
    #        if (count % 2 == 0 and count < len(players_hands)) or (count % 2 == 1 and count >= len(players_hands)):
    #            players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[0]])
    #            deck_index += distribution_pattern[0] 
    #        else:
    #            players_hands[player].append(self.deck[deck_index:deck_index+distribution_pattern[1]])
    #            deck_index += distribution_pattern[1]
        

    #    for key in players_hands.keys():
    #        players_hands[key] =  [item for sublist in players_hands[key] for item in sublist]

        return players_hands, trump_card

    def check_win(self) -> bool:
        """
        TODO
        
        Returns
        -------
        bool
            TODO
        """
        return False

    def reward(self) -> float:
        """
        TODO

        Returns
        -------
        float
            TODO
        """
        return 1

    def step(self, a) -> dict:
        """
        TODO

        Parameters
        ----------
        a : TODO
            TODO

        Returns
        -------
        dict
            TODO
        """

        return {}

In [16]:
env_config = {'score_to_win':10,'values':['9','10','J','Q','K','A']}    #set up the environmental config
euchre = Euchre(env_config)

In [ ]:
players_hands, trump = euchre.deal_cards()

['Q♣', 'K♦', 'K♠', 'Q♦', '9♠', '9♦', 'Q♠', 'K♥', '9♥', 'J♠', 'A♥', 'A♠', '10♥', 'Q♥', 'A♦', 'A♣', '10♦', 'J♥', 'K♣', 'J♦', 'J♣', '9♣', '10♣', '10♠']
{'p1': ['Q♣', '9♠', '9♥', '10♥', '10♦'], 'p2': ['K♦', '9♦', 'J♠', 'Q♥', 'J♥'], 'p3': ['K♠', 'Q♠', 'A♥', 'A♦', 'K♣'], 'p4': ['Q♦', 'K♥', 'A♠', 'A♣', 'J♦']}
J♣
